In [3]:
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime
from utils import *
import pandas as pd
from tqdm import tqdm

import tensorflow as tf

from keras.layers import Input, Embedding, LSTM, Dense, merge
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
from keras.layers import Convolution1D, GlobalMaxPooling1D, MaxPooling1D
from keras.layers import Embedding
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from IPython.display import SVG
from keras.utils.visualize_util import plot
from keras.utils.visualize_util import model_to_dot
from keras.preprocessing import text
from keras.preprocessing import sequence

from keras_tqdm import TQDMNotebookCallback

from sklearn import metrics

Using TensorFlow backend.


In [4]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [15]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

In [16]:
sentances = []

In [41]:
with open('reddit_comments.csv', 'r',  encoding="utf8") as f:
    reader = csv.reader(f, skipinitialspace=True)
    reader.__next__()
    # Split full comments into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].lower()) for x in reader])
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print("Parsed %d sentences." % (len(sentences)))

Parsed 36675 sentences.


In [42]:
sentences[0:10]

['SENTENCE_START i would actually love that. SENTENCE_END',
 'SENTENCE_START c9 are my na team and najin were the first team i bothered to watch in esports so i want them to do it. SENTENCE_END',
 'SENTENCE_START army surplus SENTENCE_END',
 'SENTENCE_START huh. SENTENCE_END',
 'SENTENCE_START never knew max was a vampire. SENTENCE_END',
 "SENTENCE_START couldn't feel more ashamed. SENTENCE_END",
 'SENTENCE_START then we wonder where these racist, ignorant cunts like the ones showing up in this thread are coming from. SENTENCE_END',
 'SENTENCE_START welcome! SENTENCE_END',
 'SENTENCE_START what is your favorite could not live without it product? SENTENCE_END',
 'SENTENCE_START i know it might grind your gears, but it currently is true. SENTENCE_END']

In [17]:
# for sentance in tqdm(list(pd.read_csv('reddit_comments.csv')['body'])):
#     sentances.append(nltk.sent_tokenize(sentance.lower()))

    
# sentances = itertools.chain(*sentances)
# sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentances]

100%|█████████████████████████████████████████████████████████████████████████| 15000/15000 [00:01<00:00, 13444.76it/s]


In [43]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

In [44]:
# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))

In [45]:
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [46]:
# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

In [47]:

# Create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

In [48]:
sentences[0:10]

['SENTENCE_START i would actually love that. SENTENCE_END',
 'SENTENCE_START c9 are my na team and najin were the first team i bothered to watch in esports so i want them to do it. SENTENCE_END',
 'SENTENCE_START army surplus SENTENCE_END',
 'SENTENCE_START huh. SENTENCE_END',
 'SENTENCE_START never knew max was a vampire. SENTENCE_END',
 "SENTENCE_START couldn't feel more ashamed. SENTENCE_END",
 'SENTENCE_START then we wonder where these racist, ignorant cunts like the ones showing up in this thread are coming from. SENTENCE_END',
 'SENTENCE_START welcome! SENTENCE_END',
 'SENTENCE_START what is your favorite could not live without it product? SENTENCE_END',
 'SENTENCE_START i know it might grind your gears, but it currently is true. SENTENCE_END']

In [51]:
np.shape(X_train)

(36675,)

In [52]:
np.shape(y_train)

(36675,)

In [49]:
X_train[1:10]

array([ [0, 4410, 31, 35, 313, 256, 8, 7999, 114, 3, 139, 256, 5, 2640, 6, 247, 14, 7999, 42, 5, 109, 72, 6, 26, 9, 2],
       [0, 1965, 7768], [0, 1668, 2], [0, 142, 688, 871, 30, 7, 7999, 2],
       [0, 104, 17, 172, 63, 4582, 2],
       [0, 96, 57, 821, 150, 165, 1102, 4, 2386, 7999, 44, 3, 379, 1270, 62, 14, 23, 483, 31, 429, 61, 2],
       [0, 1163, 33], [0, 50, 13, 41, 515, 104, 29, 345, 232, 9, 993, 18],
       [0, 5, 82, 9, 200, 5283, 41, 5742, 4, 24, 9, 644, 13, 328, 2]], dtype=object)

In [50]:
x_example, y_example = X_train[17], y_train[17]
print ("x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example))
print ("\ny:\n%s\n%s" % (" ".join([index_to_word[x] for x in y_example]), y_example))

x:
SENTENCE_START : (
[0, 20, 22]

y:
: ( SENTENCE_END
[20, 22, 1]


In [26]:
max_features = 4000
maxlen = 250
batch_size = 32
embedding_dims = 60
nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 2

In [ ]:
optimizer = RMSprop(lr=1e-4)
objective = 'binary_crossentropy'


text_seq_input =  Input(shape=(36675,), name = 'text_seq_input')

model_1 = Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen,
                    dropout=0.1)(text_seq_input)

#model.add(LSTM(32, input_shape=(10, 64)))

#model.add(LSTM(16))

model_1 = LSTM(100, return_sequences=True)(model_1)

# model_1 = MaxPooling1D(pool_length=2, border_mode='valid')(model_1)

# model_1 = Convolution1D(nb_filter=120,
#                         filter_length=4,
#                         border_mode='valid',
#                         activation='relu',
#                         subsample_length=1)(model_1)

#model = MaxPooling1D(pool_length=2, border_mode='valid')(model)

# model = Convolution1D(nb_filter=60,
#                         filter_length=5,
#                         border_mode='valid',
#                         activation='relu',
#                         subsample_length=1)(model)


#model_1 = GlobalMaxPooling1D()(model_1)
#model_1 = MaxPooling1D(pool_length=2, border_mode='valid')(model_1)


# model_1 = Dense(250, activation='relu')(model_1)
# model_1 = Dropout(0.25)(model_1)

# model_1 = Dense(50, activation='relu')(model_1)
# model_1 = Dropout(0.4)(model_1)

#model_1_out = Flatten()(model_1)
#model_1_out = Flatten()(model_1)
model = Activation('sigmoid')(model)


model = Dense(3)(model)
model = Activation('softmax')(model)


class_model = Model(input = [text_seq_input, clustered_input, text_matrix_input], output = [model, aux_1_output, aux_2_output, aux_3_output])
class_model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'], loss_weights = [0.5, 0.25, 0.25, 0.25])